In [ ]:
!pip install opendatasets

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score,precision_score,recall_score

In [ ]:
od.download('https://www.kaggle.com/competitions/nexus-by-djs-nsdc-ultraceuticals/data?select=train.csv')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: luvvswami
Your Kaggle Key: ··········


100%|██████████| 149M/149M [00:05<00:00, 27.0MB/s]



Extracting archive ./nexus-by-djs-nsdc-ultraceuticals/nexus-by-djs-nsdc-ultraceuticals.zip to ./nexus-by-djs-nsdc-ultraceuticals


In [116]:
df=pd.read_csv('/content/train.csv')

In [117]:
target_cols=['Target_Status']

In [118]:
inputs=df.drop('Target_Status',axis=1)
inputs = df.drop('ID', axis=1)
inputs=inputs.drop(target_cols,axis=1)
targets=df[target_cols]

In [119]:
print(inputs.columns)

Index(['TargetID', 'DRUGID', 'DRUGTYPE', 'Drug_high_status', 'DRUGNAME',
       'PUBCHCID', 'Disease_of_highest_status', 'Drug_Status', 'UNIPROID',
       'TARGNAME', 'GENENAME', 'SYNONYMS', 'FUNCTION', 'BIOCLASS', 'SEQUENCE',
       'Disease', 'Accession Number'],
      dtype='object')


In [120]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(inputs)
encoded_cols = encoder.transform(inputs)
encoded_cols

<134486x14331 sparse matrix of type '<class 'numpy.float64'>'
	with 2286262 stored elements in Compressed Sparse Row format>

In [121]:
x_train,x_test,y_train,y_test=train_test_split(encoded_cols,targets,test_size=0.2,random_state=42)
model=DecisionTreeClassifier(criterion='gini')
model.fit(x_train,y_train)
y_predict=model.predict(x_test)

In [122]:
accuracy=accuracy_score(y_test,y_predict)
accuracy

0.9920811956279277

In [123]:
precision=precision_score(y_test,y_predict,average='micro')
precision

0.9920811956279277

In [124]:
recall=recall_score(y_test,y_predict,average='micro')
recall

0.9920811956279277

In [125]:
f1=f1_score(y_test,y_predict,average='weighted')
f1

0.9920765559430988

In [126]:
'''
Remove ID 99.20013
Remove TargetID: 99.19654
Remove DRUGID: 99.196731

'''

' \nRemove ID 99.20013\nRemove TargetID: 99.19654\nRemove DRUGID: 99.196731\n\n'

In [145]:
test_data = pd.read_csv("/content/test.csv")
copy = test_data.copy()
test_data = test_data.drop('ID', axis=1)

In [146]:
encoded_test_cols = encoder.transform(test_data)

In [147]:
predictions = model.predict(encoded_test_cols)

In [138]:
predictions_df = pd.DataFrame({'ID': copy['ID'],'Target_Status': predictions})

In [17]:
predictions_df

,ID,Target_Status
0,1076,Approved
1,190816,Phase 3
2,180551,Phase 3
3,51630,Approved
4,50566,Phase 1/2
...,...,...
57632,33517,Phase 2
57633,177422,Phase 1/2
57634,520,Approved
57635,15371,Investigative


In [144]:
predictions_df.to_csv('predictions.csv', index=False)

In [140]:
submit_data = pd.read_csv("/content/predictions.csv")

In [141]:
print(submit_data[submit_data.duplicated(['ID'])])

Empty DataFrame
Columns: [ID, Target_Status]
Index: []


In [142]:
submit_data.drop_duplicates(subset=['ID'], inplace=True)

In [143]:
submit_data.to_csv('submission.csv', index=False)